# Tutorial: Bài toán phân loại với thuật toán Softmax Regression

Ở tutorial này, chúng ta sẽ từng bước xây dựng một mô hình máy học bằng ngôn ngữ python và thư viện tensorflow để giải quyết bài toán phân loại hoa iris.
<br>
Thuật toán được sử dụng trong bài sẽ là thuật toán softmax regression.
<br>
Đầu tiên, ta cần import các thư viện cần sử dụng:

In [1]:
import numpy as np
from sklearn.datasets import load_iris
from numpy.linalg import inv
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, StandardScaler

/Users/nhat/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/nhat/anaconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Dữ liệu mà tutorial này sử dụng sẽ là bộ dữ liệu iris. Bộ dữ liệu này gồm 150 điểm dữ liệu với 4 biến và được phân thành 3 lớp.
<br>
Bộ dữ liệu này đi kèm với thư viện scikit-learn và có thể được load như sau:

In [2]:
iris = load_iris()
X = iris['data']
y = iris['target']

Tiếp đến, chúng ta cần tiền xử lý dữ liệu. Chú ý rằng các biến của bộ dữ liệu iris đã được tiền xử lý, nên chúng ta chỉ cần tách một phần dữ liệu ra để làm dữ liệu test:

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 32)

Để giải quyết bài toán phân loại, chúng ta sẽ sử dụng thuật toán softmax regression. Đầu tiên, chúng ta sẽ sử dụng thư viện sklearn:

In [4]:
sklearn_model = LogisticRegression(solver = 'lbfgs')
sklearn_model.fit(X_train, y_train)
predictions_sklearn = sklearn_model.predict(X_test)

Chúng ta có thể kiểm tra độ chính xác của thuật toán như sau:

In [5]:
print("Accuracy for Sklearn Model:")
print(np.average(np.equal(predictions_sklearn, y_test)))

Accuracy for Sklearn Model:
0.9333333333333333


Để hiểu kỹ hơn về thuật toán, chúng ta sẽ tự lập trình thuật toán bằng thư viện tensorflow. Nhắc lại phương trình của mô hình:
$$ h(x) = \sigma(Wx + b) $$
với $\sigma$ là hàm softmax, cho bởi công thức:
$$\sigma \left(\begin{bmatrix} x_1 \\ x_2 \\x_3 \end{bmatrix} \right) = \begin{bmatrix} \frac{e^{x_1}}{e^{x_1} + e^{x_2} + e^{x_3}} \\ \frac{e^{x_2}}{e^{x_1} + e^{x_2} + e^{x_3}} \\ \frac{e^{x_3}}{e^{x_1} + e^{x_2} + e^{x_3}} \end{bmatrix}$$
$W \in \mathbb{R}^{150 \times 3}, b \in \mathbb{R}^3$ là các ma trận trọng số.
<br>
Hàm mất mát của thuật toán là hàm cross entropy:
$$ J(W, b) = -\sum_{i = 1}^N \sum_{c = 1}^3 y_{i, c} \log h(x_i)_c $$
Chúng ta sẽ implement thuật toán softmax bằng thư viện tensorflow. Hàm __init__ có tác dụng vẽ đồ thị tính toán (computational graph) của thuật toán. Hàm fit có tác dụng tối thiểu hàm loss bằng thuật toán Adam (đã được lập trình sẵn bởi thư viện tensorflow). Hàm predict dự đoán nhãn từ dữ liệu đầu vào x. Hàm evaluate đánh giá độ chính xác của mô hình với dữ liệu test.

In [6]:
class SoftmaxRegression:
    def __init__(self, n_features, n_classes):
        self._n_classes = n_classes

        self._X = tf.placeholder(shape = [None, n_features], dtype = tf.float32)
        self._W = tf.get_variable(name = "W", shape = [n_features, n_classes])
        self._b = tf.get_variable(name = "b", shape = [n_classes])

        self._z = tf.matmul(self._X, self._W) + self._b
        self._op = tf.nn.softmax(self._z)

    def fit(self, X_train, y_train, n_epochs = 10000):
        self._lb = LabelBinarizer()
        y_train_enc = self._lb.fit_transform(y_train)
        self._y = tf.placeholder(shape = [None, self._n_classes], dtype = tf.float32)
        self._loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels = self._y,
                                                                logits = self._z)

        self._optimizer = tf.train.AdamOptimizer()
        train_step = self._optimizer.minimize(self._loss)

        self._sess = tf.Session()
        self._sess.run(tf.global_variables_initializer())

        for _ in range(n_epochs):
            self._sess.run(train_step, feed_dict = {self._X: X_train,
                                                    self._y: y_train_enc})


    def predict(self, X_test):
        proba = self._sess.run(self._op, feed_dict = {self._X: X_test})
        return np.argmax(proba, axis = -1)

    def evaluate(self, X_test, y_test):
        y_test_enc = self._lb.transform(y_test)
        accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(self._y, axis=-1),
                                                   tf.argmax(self._op, axis=-1)), tf.float32))
        return self._sess.run(accuracy, feed_dict = {self._X: X_test, self._y: y_test_enc})

Bây giờ chúng ta sẽ thử tính độ chính xác của mô hình:

In [7]:
tf_model = SoftmaxRegression(n_features = 4, n_classes = 3)
tf_model.fit(X_train, y_train)
print(tf_model.evaluate(X_test, y_test))
print(tf_model.evaluate(X, y))

1.0
0.98
